# Logistic Regression

In [65]:
import scipy.io as sio
from sklearn.preprocessing import normalize
from sklearn import linear_model as lm
import pandas as pd
import csv
import numpy as np

In [81]:
data_path = 'classifier.mat'

data = sio.loadmat(data_path)
org_account_type = data['training_labels'].T
org_account_data = data['training_data'] 
indiv_account_data = data['individual_data'] 
indiv_account_order = data['individual_account_order'] 

In [82]:
data.keys()

dict_keys(['training_labels', 'individual_account_order', 'individual_data', '__header__', 'training_data', '__version__', '__globals__'])

In [83]:
org_account_type.shape

(80, 1)

In [84]:
indiv_account_data.shape

(200, 112)

----
# Build Classifier

In [85]:
# Normalize the data set
normed_org_acnt_data = normalize(org_account_data, axis=1, norm='l2')
normed_indiv_data = normalize(indiv_account_data, axis=1, norm='l2')

In [86]:
# Fit the model to logistic function
logistic = lm.LogisticRegression(fit_intercept=True)
logistic.fit(normed_org_acnt_data, org_account_type)
print('LogisticRegression score: %f'
      % logistic.fit(normed_org_acnt_data, org_account_type).score(normed_org_acnt_data, org_account_type))

LogisticRegression score: 0.975000


/Users/juntan/anaconda3/envs/datascience/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [87]:
logistic.coef_

array([[  1.67463218e+00,   1.78919596e+00,  -8.53989161e-01,
          7.19771160e-01,   8.16356455e-01,   7.45357062e-01,
         -1.79027230e+00,   5.97134461e-01,   5.54541444e-01,
         -2.17075920e-01,   4.24793956e-01,   3.79094695e-01,
         -1.23285504e-01,   2.89837883e-01,   1.33540945e-02,
          3.24532032e-01,  -1.36818643e-01,   3.44213083e-01,
         -2.33203825e-01,   3.95548506e-01,  -3.47542946e-03,
         -1.14792640e-01,  -4.39516228e-01,   1.86079037e-01,
         -1.44651625e-01,   3.15233839e-01,   1.40732485e-01,
          1.24816098e-01,   3.55312923e-02,   1.96123980e-01,
          1.95052035e-01,  -1.78062551e-01,   4.24504157e-01,
         -5.61924219e-02,  -2.20821953e-01,  -3.11060240e-01,
         -3.89779025e-01,  -1.08198869e-03,  -8.42577604e-02,
         -3.00186282e-01,  -9.98429291e-02,  -2.68440158e-01,
          9.23430504e-02,   1.99518729e-01,  -2.44903172e-01,
          7.41746479e-02,   2.07270699e-01,  -1.29821931e-01,
        

In [88]:
np.argmax(logistic.coef_)

1

Once we fit the logistic model to the hand labeled popular accounts, we can use the logistic model to predict the type of the individual accounts and the strength of the belief. $logistic.predict$ produces the probability of the sample for each class in the model, where classes are ordered as they are in logistic.classes_ = array([0, 1]), 1 is prlife, 0 is prochoice

In [89]:
# Calculate the strength of the belief for individual accounts
strengths = logistic.predict_proba(normed_indiv_data)

In [90]:
pred = logistic.predict(normed_indiv_data)

In [91]:
file_dict = {}
file_dict['individual_account_order'] = indiv_account_order
file_dict['prochoice_strength'] = strengths[:,0]
file_dict['prolife_strength'] = strengths[:,1]
file_dict['predict_class'] = pred
sio.savemat('individual_strength_class.mat', file_dict, do_compression=True)

In [92]:
logistic.classes_

array([0, 1])

# Combine result from following diversity

In [31]:
classifier_df = pd.DataFrame(
    {
    "account_name": [s.strip() for s in indiv_account_order], 
    "prochoice_strength": strengths[:,0],
    "prolife_strength": strengths[:,1],
    "predict_class": pred
    }
)

classifier_df.head()

,account_name,predict_class,prochoice_strength,prolife_strength
0,ABabysRights,1,0.112896,0.887104
1,ABGYNservices,0,0.541203,0.458797
2,Abigailschumach,1,0.305871,0.694129
3,Alboteaparty,0,0.561752,0.438248
4,alexdelong_,1,0.141862,0.858138


There are 197 accounts in the result_df.

In [32]:
classifier_df.shape

(200, 4)

Add a column tells whether the user is a follower of the prolife account (1) or prochoice (-1) account or both (0).

In [33]:
prolife_followers = []
with open('./prolife_followers.csv', 'r') as f:
    reader = csv.reader(f)
    for item in list(reader):
        prolife_followers += item
assert len(set(prolife_followers))==100

prochoice_followers = []
with open('./prochoice_followers.csv', 'r') as f:
    reader = csv.reader(f)
    for item in list(reader):
        prochoice_followers += item
assert len(set(prochoice_followers))==100

classifier_df['follower of'] = [0 if user in prolife_followers and user in prochoice_followers 
                            else 1 if user in prolife_followers else -1 for user in classifier_df['account_name']]

Combine the classifier's result with the following diversity result.

In [34]:
following_proportion = pd.read_csv('proportion.csv')
combine_df = pd.merge(classifier_df, following_proportion, left_on='account_name', right_on='User', how='left')
result_df = combine_df.drop_duplicates()
result_df = result_df.drop('User', axis=1)
print("size of the data frame", result_df.shape)
result_df.tail()

size of the data frame (200, 7)


,account_name,predict_class,prochoice_strength,prolife_strength,follower of,prolife proportion,prochoice proportion
195,VLabarum,1,0.440136,0.559864,1,1.0,0.0
196,wafulkerson1,0,0.609548,0.390452,1,1.0,0.0
197,WarrenDavidson,1,0.270500,0.729500,1,0.9,0.1
198,WilsonRx,1,0.351691,0.648309,1,1.0,0.0
199,wordspurify27,0,0.578185,0.421815,-1,0.0,1.0


# Save the file

In [35]:
result_df.to_csv("hashtag_indiv_strength_diversity.csv", index=False)